# Amazon Review Analysis

This is an analysis of the Amazon pet product reviews data. The goal is to determine whether or not there was any bias for reviews that were written as part of the Vine program. We will determine if having paid Vine reviews makes a difference in the percentage of 5-star reviews.

### Dependencies and Data

In [1]:
# Locate Spark
import findspark
findspark.init()

# Dependencies
from pyspark import SparkFiles
from pyspark.sql import SparkSession

# Spark session adding the Postgres driver to Spark
spark = SparkSession.builder.appName('amz').getOrCreate()
spark

In [2]:
# Read in data
df = spark.read.csv('pet_product_reviews.tsv', sep='\t', header=True, inferSchema=True)
print(df.count())
df.show(1, vertical=True, truncate=False)

2643619
-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 marketplace       | US                                                                                                                                                                                                                                                           
 customer_id       | 28794885                                                                                                                                                                                                                                                     
 review_id         | REAKC26P07MDN                                                                                                                                     

### Vine Reviews vs. Non-vine Reviews

1. Filter for rows where:
    - `total_votes` >= 5 (at least 5 votes)
    - `helpful_votes` / `total_votes` >= 0.2 (at least 20% of the votes are helpful)
2. Split the filtered data into `vine` reviews and non-`vine` reviews
3. Compare the percentage of 5-star reviews from the 2 groups

In [3]:
# Filter data
filtered_df = df.filter(df['total_votes'] >= 5)
filtered_df = filtered_df.filter(df['helpful_votes'] / df['total_votes'] >= 0.2)
filtered_df.count()

196299

In [ ]:
# Vine reviews
vine_df = filtered_df.filter(filtered_df['vine'] == 'Y')
vine_count = vine_df.count()

# Non-vine reviews
nonvine_df = filtered_df.filter(filtered_df['vine'] == 'N')
nonvine_count = nonvine_df.count()

vine_count, nonvine_count

In [ ]:
# 5-star count for vine reviews
vine5_df = vine_df.filter(vine_df['star_rating'] == 5)
vine5_count = vine5_df.count()

# 5-star count for non-vine reviews
nonvine5_df = nonvine_df.filter(nonvine_df['star_rating'] == 5)
nonvine5_count = nonvine5_df.count()

vine5_count, nonvine5_count

In [ ]:
# 5-star percentages
print('Vine 5-star review %:', vine5_count / vine_count * 100)
print('Non-vine 5-star review %:', nonvine5_count / nonvine_count * 100)

### Comparison with Equal Sampling

The comparison above is extremely unfair as there were 195,547 reviews in the `non-vine` group and only 752 reviews in the `vine` group (less than 1% of the reviews). So we will now use random sampling with replacement to produce 2 equal groups and make the comparison again.

The filtered data has 195,547 rows, so we will be taking a sample of 10,000 (approximately 5%) for each group. Since the `vine` group only has 752, the sampling will be done with replacement.

In [ ]:
# Vine sample
vine_sample = vine_df.sample(fraction=(10000/vine_count), withReplacement=True, seed=0).limit(10000)
vine_sample_count = vine_sample.count()

# Non-vine sample
nonvine_sample = nonvine_df.sample(fraction=(10000/nonvine_count), withReplacement=True, seed=0).limit(10000)
nonvine_sample_count = nonvine_sample.count()

vine_sample_count, nonvine_sample_count

In [ ]:
# 5-star count for vine sample
vine5_sample = vine_sample.filter(vine_sample['star_rating'] == 5)
vine5_sample_count = vine5_sample.count()

# 5-star count for non-vine sample
nonvine5_sample = nonvine_sample.filter(nonvine_sample['star_rating'] == 5)
nonvine5_sample_count = nonvine5_sample.count()

vine5_sample_count, nonvine5_sample_count

In [ ]:
# 5-star percentages
print('Vine 5-star review %:', vine5_sample_count / vine_sample_count * 100)
print('Non-vine 5-star review %:', nonvine5_sample_count / nonvine_sample_count * 100)

### Conclusion

It doesn't look like there is a bias towards 5-star reviews with paid reviews from the Vine program. Even with the sampling method, which evened out the 2 group sizes for a fair comparison, the percentage of 5-star reviews was consistent for both groups: ~37% for the `vine` group and ~50% for the `non-vine` group.